In [3]:
import re
import pathlib
import pandas as pd

In [8]:
def extract_train_msg(test_msg):
    msg_split = re.split('save_every_n', test_msg)[1:]
    df = pd.DataFrame(columns=['seed', 'pretrained_ckpt', 'experiment', 'n_param', 'n_epoch', 'ade', 'fde'])
    for msg in msg_split: 
        metric = re.search("Round 0: \nTest ADE: ([\d\.]+) \nTest FDE: ([\d\.]+)", msg)
        n_epoch = re.search("Early stop at epoch ([\d]+)", msg)
        df = pd.concat([df, pd.DataFrame({
            'seed': re.search("'seed': ([\d+]),", msg).group(1),
            'pretrained_ckpt': re.search("'pretrained_ckpt': '(.*?)',", msg).group(1).split('/')[1],
            'experiment': re.search("Experiment (.*?) has started", msg).group(1),
            'n_param': re.search("The number of trainable parameters: ([\d]+)", msg).group(1),
            'n_epoch': n_epoch.group(1) if n_epoch is not None else 99,
            'ade': metric.group(1), 
            'fde': metric.group(2)}, index=[0])], ignore_index=True)
    df.seed = df.seed.astype(int)
    df.n_param = df.n_param.astype(int)
    df.n_epoch = df.n_epoch.astype(int)
    df.ade = df.ade.astype(float)
    df.fde = df.fde.astype(float)
    df['train_net'] = df['experiment'].apply(lambda x: get_train_net(x))
    df['n_train'] = df['experiment'].apply(lambda x: get_n_train(x)).astype(int)
    df['adapter_position'] = df['experiment'].apply(lambda x: get_adapter_position(x))
    # reorder columns 
    reordered_cols = ['seed', 'train_net', 'n_train', 'adapter_position', 'n_param', 'n_epoch', 'ade', 'fde', 'experiment', 'pretrained_ckpt']
    df = df.reindex(columns=reordered_cols)
    return df

def extract_test_msg(test_msg):
    msg_split = re.split('save_every_n', test_msg)[1:]
    df = pd.DataFrame(columns=['seed', 'pretrained_ckpt', 'tuned_ckpt', 'ade', 'fde'])
    for msg in msg_split: 
        metric = re.search("Round 0: \nTest ADE: ([\d\.]+) \nTest FDE: ([\d\.]+)", msg)
        df = pd.concat([df, pd.DataFrame({
            'seed': re.search("'seed': ([\d+]),", msg).group(1),
            'pretrained_ckpt': re.search("'pretrained_ckpt': '(.*?)',", msg).group(1).split('/')[1],
            'tuned_ckpt': re.search("'tuned_ckpt': '(.*?)',", msg).group(1).split('/')[1],
            'ade': metric.group(1), 
            'fde': metric.group(2)}, index=[0])], ignore_index=True)
    df.seed = df.seed.astype(int)
    df.ade = df.ade.astype(float)
    df.fde = df.fde.astype(float)
    df['train_net'] = df['tuned_ckpt'].apply(lambda x: get_train_net(x))
    df['n_train'] = df['tuned_ckpt'].apply(lambda x: get_n_train(x)).astype(int)
    df['adapter_position'] = df['tuned_ckpt'].apply(lambda x: get_adapter_position(x))
    # reorder columns 
    reordered_cols = ['seed', 'train_net', 'n_train', 'adapter_position', 'ade', 'fde', 'tuned_ckpt', 'pretrained_ckpt']
    df = df.reindex(columns=reordered_cols)
    return df

def get_train_net(ckpt_path):
    return ckpt_path.split('__')[5]

def get_n_train(ckpt_path):
    if 'adapter' in ckpt_path:
        n_train = ckpt_path.split('__')[7].split('_')[1].split('.')[0]
    elif 'weight' in ckpt_path:
        n_train = ckpt_path.split('__')[6].split('_')[1]
    else:
        n_train = ckpt_path.split('__')[6].split('_')[1].split('.')[0]
    return n_train

def get_adapter_position(ckpt_path):
    if 'adapter' in ckpt_path:
        return ckpt_path.split('__')[6]
    else:
        return None 

In [19]:
def extract_file(file_path, out_dir='../csv/log'):
    with open(file_path, 'r') as f:
        msgs = f.read()
    if 'eval' in file_path:
        df = extract_test_msg(msgs)
    else: # train
        df = extract_train_msg(msgs)
    pathlib.Path(out_dir).mkdir(parents=True, exist_ok=True)
    file_name = re.search('/([^/]+).out', file_path).group(1)
    out_name = f'{out_dir}/{file_name}.csv'
    print(out_name)
    df.to_csv(out_name, index=False)


In [155]:
extract_file('../DC_TrN_80_encoder_i_eval.out')

../csv/log/DC_TrN_80_encoder_i_eval.csv


In [20]:
extract_file('../logs/DC_TrN_80_encoder_i_train.out')

DC_TrN_80_encoder_i_train
../csv/log/DC_TrN_80_encoder_i_train.csv


In [41]:
imp_msg = '''
{'save_every_n': 10, 'n_early_stop': 10, 'resize_factor': 0.25, 'viz_epoch': 10, 'encoder_channels': [32, 32, 64, 64, 64], 'decoder_channels': [64, 64, 64, 32, 32], 'waypoints': [11], 'temperature': 1.0, 'n_semantic_classes': 6, 'loss_scale': 1000, 'kernlen': 31, 'nsig': 4, 'use_features_only': False, 'e_unfreeze': 150, 'use_TTST': False, 'rel_threshold': 0.01, 'use_CWS': False, 'CWS_params': 'None', 'obs_len': 8, 'pred_len': 12, 'n_goal': 20, 'n_traj': 1, 'use_raw_data': True, 'data_dir': '/data/dli-data/', 'dataset_name': 'sdd', 'out_csv_dir': 'csv', 'segmentation_model_fp': '/data/dli-data/sdd/segmentation_model.pth', 'dataset_path': 'filter/agent_type/deathCircle_0/', 'val_files': ['Biker.pkl'], 'val_ratio': 0.1, 'n_leftouts': [500], 'share_val_test': True, 'ckpts': None, 'ckpts_name': None, 'pretrained_ckpt': 'ckpts/Seed_1__Tr_Pedestrian__Val_Pedestrian__ValRatio_0.1__filter_agent_type__train.pt', 'tuned_ckpt': None, 'tuned_ckpts': ['ckpts/DC0__lora/Seed_1__Tr_Biker__Val_Biker__ValRatio_0.1__filter_agent_type_deathCircle_0__lora_1__Pos_0_1_2_3_4__TrN_20__lr_0.0005.pt'], 'position': [], 'ynet_bias': False, 'train_net': 'all', 'seed': 1, 'batch_size': 8, 'gpu': None, 'n_round': 1, 'config_filename': 'sdd_raw_eval.yaml', 'given_meta_ids': None, 'result_path': None, 'result_name': None, 'result_limited': None, 'random_n': None}
Split ['Biker.pkl'] given val_ratio=0, n_leftout=[500]
df_test: (10000, 10); #=500.0
Focusing on meta_ids= [5152 5175 5176 5177 5178 5179 5180 5181 5182 5183 5184 5185 5186 5187
 5188 5189 5190 5191 5194 5195 5196 5197 5198 5199 5200 5201 5202 5203
 5204 5205 5214 5215 5220 5229 5230 5231 5232 5233 5234 5235 5236 5237
 5238 5240 5241 5242 5243 5246 5248 5249 5250 5251 5252 5253 5256 5257
 5258 5259 5260 5261 5262 5263 5264 5265 5266 5267 5268 5269 5285 5286
 5287 5288 5299 5300 5301 5302 5303 5304 5305 5306 5307 5315 5316 5317
 5318 5323 5331 5332 5333 5334 5335 5337 5341 5342 5355 5356 5357 5358
 5359 5360 5361 5362 5375 5376 5377 5378 5379 5384 5385 5386 5387 5388
 5389 5390 5399 5409 5436 5444 5445 5446 5447 5448 5449 5450 5451 5452
 5457 5458 5459 5460 5461 5462 5463 5464 5465 5466 5467 5468 5469 5470
 5471 5472 5473 5474 5475 5476 5477 5478 5507 5508 5509 5510 5511 5512
 5513 5605 5606 5607 5608 5609 5610 5611 5612 5613 5614 5615 5616 5617
 5621 5622 5623 5624 5634 5635 5636 5637 5638 5639 5640 5641 5642 5643
 5644 5645 5646 5647 5648 5649 5650 5651 5652 5653 5654 5655 5656 5657
 5658 5659 5660 5661 5662 5663 5664 5665 5666 5667 5668 5669 5670 5671
 5672 5673 5674 5675 5676 5677 5678 5679 5680 5686 5687 5688 5689 5690
 5691 5692 5693 5694 5695 5696 5697 5698 5699 5710 5711 5714 5715 5718
 5719 5720 5721 5722 5725 5726 5727 5729 5730 5731 5732 5738 5748 5749
 5750 5751 5752 5753 5754 5755 5756 5757 5758 5760 5761 5762 5763 5766
 5767 5768 5769 5770 5771 5772 5774 5775 5776 5777 5778 5781 5793 5794
 5797 5798 5799 5800 5801 5802 5803 5804 5805 5806 5807 5808 5809 5810
 5811 5812 5813 5814 5815 5816 5817 5818 5819 5820 5821 5838 5839 5840
 5841 5842 5843 5844 5850 5851 5852 5853 5854 5855 5856 5857 5858 5859
 5860 5876 5877 5878 5879 5880 5881 5882 5883 5884 5885 5886 5887 5888
 5889 5890 5891 5905 5906 5931 5932 5933 5934 5935 5945 5946 5947 5948
 5949 5950 5951 5952 5953 5954 5955 5956 5957 5958 5959 5964 5965 5966
 5967 5968 5969 5970 5971 5972 5976 5977 5978 5979 5980 5981 5982 5991
 5992 6053 6054 6055 6060 6061 6062 6063 6066 6067 6068 6069 6070 6075
 6076 6077 6084 6085 6086 6087 6097 6098 6099 6100 6101 6104 6105 6113
 6114 6115 6116 6117 6118 6133 6134 6135 6153 6154 6155 6160 6161 6162
 6163 6164 6165 6166 6167 6168 6173 6174 6186 6187 6189 6190 6191 6192
 6205 6206 6207 6208 6209 6210 6211 6212 6213 6214 6222 6223 6227 6228
 6229 6230 6231 6232 6233 6234 6235 6236 6240 6241 6249 6251 6252 6259
 6260 6261 6268 6269 6270 6279 6280 6281 6282 6283 6288 6301 6302 6303
 6304 6305 6306 6307 6308 6309 6310 6311 6312 6313 6314 6315 6316 6317
 6318 6319 6320 6321 6322 6323 6324 6325 6326 6327]
df_test_limited: (10000, 10); #=500.0
Working on cuda
Loaded segmentation model to GPU
Loaded ynet model to GPU
Replacing encoder.stages.0.0.lora_A
Working on cuda
Loaded segmentation model to GPU
Loaded ynet model to GPU
No data and images augmentation
TTST setting: False
Start testing
Round 0: 
Test ADE: 34.598594665527344 
Test FDE: 59.70595932006836

Average performance (by 1): 
Test ADE: 34.598594665527344 
Test FDE: 59.70595932006836
Replacing encoder.stages.1.1.lora_A
Working on cuda
Loaded segmentation model to GPU
Loaded ynet model to GPU
No data and images augmentation
TTST setting: False
Start testing
Round 0: 
Test ADE: 31.059904098510742 
Test FDE: 53.201873779296875

Average performance (by 1): 
Test ADE: 31.059904098510742 
Test FDE: 53.201873779296875
Replacing encoder.stages.1.3.lora_A
Working on cuda
Loaded segmentation model to GPU
Loaded ynet model to GPU
No data and images augmentation
TTST setting: False
Start testing
Round 0: 
Test ADE: 31.024852752685547 
Test FDE: 53.47703552246094

Average performance (by 1): 
Test ADE: 31.024852752685547 
Test FDE: 53.47703552246094'''

In [46]:
msg_split = re.split('save_every_n', imp_msg)[1:]
df = pd.DataFrame(columns=['seed', 'layer', 'ade', 'fde', 'tuned_ckpt', 'pretrained_ckpt'])
for msg in msg_split: 
    seed = re.search("'seed': ([\d+]),", msg)
    pretrained_ckpt = re.search("'pretrained_ckpt': '(.*?)',", msg)
    tuned_ckpt = re.search("'tuned_ckpts': \['(.*?)'\],", msg)
    layers = re.findall("Replacing (.*?)\n", msg)
    metrics = re.findall("Round 0: \nTest ADE: ([\d\.]+) \nTest FDE: ([\d\.]+)", msg)
    print(metrics)
    # temp
    df_temp = pd.DataFrame()
    df_temp['layer'] = layers 
    df_temp['metric'] = metrics
    df_temp['ade'] = df_temp.metric.apply(lambda x: x[0])
    df_temp['fde'] = df_temp.metric.apply(lambda x: x[1])
    df_temp['seed'] = seed.group(1) if seed is not None else None 
    df_temp['tuned_ckpt'] = tuned_ckpt.group(1) if tuned_ckpt is not None else None 
    df_temp['pretrained_ckpt'] = pretrained_ckpt.group(1) if pretrained_ckpt is not None else None 
    df = pd.concat([df, df_temp], axis=0, ignore_index=True)
    df.drop(columns=['metric'], inplace=True)

[('34.598594665527344', '59.70595932006836'), ('31.059904098510742', '53.201873779296875'), ('31.024852752685547', '53.47703552246094')]


In [47]:
df

,seed,layer,ade,fde,tuned_ckpt,pretrained_ckpt
0,1,encoder.stages.0.0.lora_A,34.598594665527344,59.70595932006836,ckpts/DC0__lora/Seed_1__Tr_Biker__Val_Biker__V...,ckpts/Seed_1__Tr_Pedestrian__Val_Pedestrian__V...
1,1,encoder.stages.1.1.lora_A,31.059904098510742,53.201873779296875,ckpts/DC0__lora/Seed_1__Tr_Biker__Val_Biker__V...,ckpts/Seed_1__Tr_Pedestrian__Val_Pedestrian__V...
2,1,encoder.stages.1.3.lora_A,31.024852752685547,53.47703552246094,ckpts/DC0__lora/Seed_1__Tr_Biker__Val_Biker__V...,ckpts/Seed_1__Tr_Pedestrian__Val_Pedestrian__V...


In [35]:
df = pd.DataFrame()
df['ade'] = metrics

In [36]:
df.apply(lambda x: x[0])

,ade
0,34.598594665527344
1,59.70595932006836


## Rename ckpts

In [1]:
import os 

In [8]:
old_name = '../ckpts/Seed_1__Tr_Biker__Val_Biker__ValRatio_0.1__filter_agent_type_deathCircle_0__adapter_parallel__0__TrN_10__AUG.pt'
new_name = old_name
new_name = new_name.replace('adapter_', '')
new_name = new_name.replace('.pt', '__lr_0.00005.pt')
name_split = new_name.split('__')
name_split[6] = 'Pos_' + name_split[6]
new_name = '__'.join(name_split)
print(old_name, new_name)

../ckpts/Seed_1__Tr_Biker__Val_Biker__ValRatio_0.1__filter_agent_type_deathCircle_0__adapter_parallel__0__TrN_10__AUG.pt ../ckpts/Seed_1__Tr_Biker__Val_Biker__ValRatio_0.1__filter_agent_type_deathCircle_0__parallel__Pos_0__TrN_10__AUG__lr_0.00005.pt


In [10]:
os.rename(old_name, new_name)

In [18]:
old_name = 'Seed_1__Tr_Biker__Val_Biker__ValRatio_0.1__filter_agent_type_deathCircle_0__encoder_0-2__TrN_80.pt'
encoder_dict={
    'all_FT': 'all', 'encoder_0': 'encoder__Pos_0', 
    'encoder_1': 'encoder__Pos_1', 'encoder_2': 'encoder__Pos_2', 
    'encoder_3': 'encoder__Pos_3', 'encoder_4': 'encoder__Pos_4',
    'encoder_0-1': 'encoder__Pos_0_1', 'encoder_0-2': 'encoder__Pos_0_1_2',
    'encoder_0-3': 'encoder__Pos_0_1_2_3', 'encoder_0-4': 'encoder__Pos_0_1_2_3_4',
    'encoder_1-2': 'encoder__Pos_1_2', 'encoder_1-3': 'encoder__Pos_1_2_3',
    'encoder_1-4': 'encoder__Pos_1_2_3_4', 'encoder_2-3': 'encoder__Pos_2_3',
    'encoder_2-4': 'encoder__Pos_2_3_4', 'encoder_3-4': 'encoder__Pos_3_4'}
new_name = old_name
encoder_i = new_name.split('__')[5]
new_name = new_name.replace(encoder_i, encoder_dict[encoder_i])
print(old_name, new_name)


Seed_1__Tr_Biker__Val_Biker__ValRatio_0.1__filter_agent_type_deathCircle_0__encoder_0-2__TrN_80.pt Seed_1__Tr_Biker__Val_Biker__ValRatio_0.1__filter_agent_type_deathCircle_0__encoder__Pos_0_1_2__TrN_80.pt


In [21]:
old_name='Seed_1__Tr_Biker__Val_Biker__ValRatio_0.1__filter_agent_type_deathCircle_0__serial__Pos_1_2_3_4__TrN_40__AUG__lr_0.00005.pt'
left=old_name.split('__AUG__')[0]
right=old_name.split('__AUG__')[1].replace('.pt', '')
new_name=f'{left}__{right}__AUG.pt'
print(old_name, new_name)

Seed_1__Tr_Biker__Val_Biker__ValRatio_0.1__filter_agent_type_deathCircle_0__serial__Pos_1_2_3_4__TrN_40__AUG__lr_0.00005.pt Seed_1__Tr_Biker__Val_Biker__ValRatio_0.1__filter_agent_type_deathCircle_0__serial__Pos_1_2_3_4__TrN_40__lr_0.00005__AUG.pt


In [23]:
old_name='Seed_1__Tr_Biker__Val_Biker__ValRatio_0.1__filter_agent_type_deathCircle_0__serial__Pos_1_2_3_4__TrN_40__lr_5e-05.pt'
new_name=old_name.replace('5e-05', '0.00005')
print(old_name, new_name)

Seed_1__Tr_Biker__Val_Biker__ValRatio_0.1__filter_agent_type_deathCircle_0__serial__Pos_1_2_3_4__TrN_40__lr_5e-05.pt Seed_1__Tr_Biker__Val_Biker__ValRatio_0.1__filter_agent_type_deathCircle_0__serial__Pos_1_2_3_4__TrN_40__lr_0.00005.pt


In [24]:
for old_name in os.listdir('../ckpts/'):
    if '5e-05' in old_name:
        left=old_name.split('__AUG__')[0]
        right=old_name.split('__AUG__')[1].replace('.pt', '')
        new_name=f'{left}__{right}__AUG.pt'
        os.rename(f'../ckpts/{old_name}', f'../ckpts/{new_name}')

In [2]:
for old_name in os.listdir('../ckpts/'):
    if 'parallelLayer' in old_name:
        new_name=old_name.replace('parallelLayer', 'parallelBlock')
        os.rename(f'../ckpts/{old_name}', f'../ckpts/{new_name}')

In [ ]:
root_dir = '../ckpts/DC0__parallelBlock'
for old_name in os.listdir(root_dir):
    if '5e-05' in old_name:
        new_name=old_name.replace('5e-05', '0.00005')
        os.rename(f'{root_dir}/{old_name}', f'{root_dir}/{new_name}')

In [2]:
for old_name in os.listdir('../noearly_shuffle/'):
    new_name=old_name.replace('___', '__')
    os.rename(f'../noearly_shuffle/{old_name}', f'../noearly_shuffle/{new_name}')